In [ ]:
#export
import csv
import numpy as np  # http://www.numpy.org
import ast
from datetime import datetime
from math import log, floor, ceil
import random

Modify the Utility class's methods. You can also add additional methods as required but don't change existing methods' arguments.

In [ ]:
#export
import math

class Utility(object):
    
    # This method computes entropy for information gain
    def entropy(self, class_y):
        # Input:            
        #   class_y         : list of class labels (0's and 1's)

        # Compute the number of data points
        total_samples = len(class_y)
        
        # Count the number of samples in each class (0 and 1)
        count_class_0 = class_y.count(0)
        count_class_1 = class_y.count(1)
        
        # Calculate the proportions of each class
        p_class_0 = count_class_0 / total_samples
        p_class_1 = count_class_1 / total_samples
        
        # Initialize entropy
        entropy = 0
        
        # Calculate entropy for each class, avoiding log(0)
        if p_class_0 > 0:
            entropy -= p_class_0 * math.log2(p_class_0)
        if p_class_1 > 0:
            entropy -= p_class_1 * math.log2(p_class_1)
        
        return entropy

    def partition_classes(self, X, y, split_attribute, split_val):
        # Inputs:
        #   X               : data containing all attributes
        #   y               : labels
        #   split_attribute : column index of the attribute to split on
        #   split_val       : a numerical value to divide the split_attribute

        # Initialize lists to hold the partitions
        X_left = []
        X_right = []

        y_left = []
        y_right = []

        # Iterate through each data point and its label
        for i in range(len(X)):
            # Check the split attribute of the current data point
            if X[i][split_attribute] <= split_val:
                # If it's less than or equal to the split value, add it to the left partition
                X_left.append(X[i])
                y_left.append(y[i])
            else:
                # Otherwise, add it to the right partition
                X_right.append(X[i])
                y_right.append(y[i])

        return (X_left, X_right, y_left, y_right)


    def information_gain(self, previous_y, current_y):
        # Calculate the entropy of the original dataset
        entropy_D = self.entropy(previous_y)

        # Initialize information gain
        info_gain = entropy_D

        # Calculate the weighted sum of entropies for each partition
        total_samples = len(previous_y)
        
        for y_partition in current_y:
            partition_weight = len(y_partition) / total_samples
            
            # Check if there are samples in the partition
            if len(y_partition) > 0:
                entropy_partition = self.entropy(y_partition)
                info_gain -= partition_weight * entropy_partition

        return info_gain


    def best_split(self, X, y):
        # Initialize variables to track the best split
        best_split_attribute = None
        best_split_val = None
        best_info_gain = -1  # Initialize with a negative value

        # Initialize empty lists for the best splits
        best_X_left = []
        best_X_right = []
        best_y_left = []
        best_y_right = []

        # Calculate the starting entropy
        starting_entropy = self.entropy(y)

        # Iterate over each attribute
        for split_attribute in range(len(X[0])):
            # Get the unique values for the current attribute
            attribute_values = set(x[split_attribute] for x in X)

            # Iterate over each value as a potential split value
            for split_val in attribute_values:
                # Partition the data based on the current attribute and value
                X_left, X_right, y_left, y_right = self.partition_classes(X, y, split_attribute, split_val)

                # Calculate information gain for this split
                current_y = [y_left, y_right]
                info_gain = self.information_gain(y, current_y)

                # Update the best split if the current split is better
                if info_gain > best_info_gain:
                    best_info_gain = info_gain
                    best_split_attribute = split_attribute
                    best_split_val = split_val
                    best_X_left = X_left
                    best_X_right = X_right
                    best_y_left = y_left
                    best_y_right = y_right

        # Create a dictionary with the best split information
        best_split_info = {
            'split_attribute': best_split_attribute,
            'split_val': best_split_val,
            'X_left': best_X_left,
            'X_right': best_X_right,
            'y_left': best_y_left,
            'y_right': best_y_right,
            'info_gain': best_info_gain
        }

        return best_split_info

### Define the classes 'DecisionTree' and 'RandomForest'

Please modify the 'DecisionTree' and 'RandomForest' classes below

In [ ]:
#export
class DecisionTree(object):
    def __init__(self, max_depth):
        # Initializing the tree as an empty dictionary or list, as preferred
        self.tree = {}
        self.max_depth = max_depth
        
    	
    def learn(self, X, y, par_node=None, depth=0):
        if depth == self.max_depth or len(set(y)) == 1:
            self.tree = {'leaf': True, 'class': max(set(y), key=y.count)}
            return

        if not X:
            self.tree = {'leaf': True, 'class': max(set(y), key=y.count)}
            return

        utility = Utility()
        split_info = utility.best_split(X, y)

        if not split_info:
            self.tree = {'leaf': True, 'class': max(set(y), key=y.count)}
            return

        X_left, X_right, y_left, y_right = split_info['X_left'], split_info['X_right'], split_info['y_left'], split_info['y_right']
        self.tree = {'leaf': False, 'split_attribute': split_info['split_attribute'], 'split_val': split_info['split_val']}
        self.tree['left'] = DecisionTree(self.max_depth)
        self.tree['right'] = DecisionTree(self.max_depth)
        self.tree['left'].learn(X_left, y_left, par_node=split_info, depth=depth + 1)
        self.tree['right'].learn(X_right, y_right, par_node=split_info, depth=depth + 1)

    def classify(self, record):
        if self.tree.get('leaf'):
            return self.tree['class']
        split_attribute, split_val = self.tree['split_attribute'], self.tree['split_val']
        if record[split_attribute] <= split_val:
            return self.tree['left'].classify(record)
        else:
            return self.tree['right'].classify(record)

In [ ]:
#export
# This starter code does not run. You will have to add your changes and
# turn in code that runs properly.

"""
Here, 
1. X is assumed to be a matrix with n rows and d columns where n is the
number of total records and d is the number of features of each record. 
2. y is assumed to be a vector of labels of length n.
3. XX is similar to X, except that XX also contains the data label for each
record.
"""

"""
This skeleton is provided to help you implement the assignment.You must 
implement the existing functions as necessary. You may add new functions
as long as they are called from within the given classes. 

VERY IMPORTANT!
Do NOT change the signature of the given functions.
Do NOT change any part of the main function APART from the forest_size parameter.  
"""


class RandomForest(object):
    num_trees = 0
    decision_trees = []

    # the bootstrapping datasets for trees
    # bootstraps_datasets is a list of lists, where each list in bootstraps_datasets is a bootstrapped dataset.
    bootstraps_datasets = []

    # the true class labels, corresponding to records in the bootstrapping datasets
    # bootstraps_labels is a list of lists, where the 'i'th list contains the labels corresponding to records in
    # the 'i'th bootstrapped dataset.
    bootstraps_labels = []

    def __init__(self, num_trees):
        # Initialization done here
        self.num_trees = num_trees
        self.decision_trees = [DecisionTree(max_depth=10) for i in range(num_trees)]
        self.bootstraps_datasets = []
        self.bootstraps_labels = []
        
    def _bootstrapping(self, XX, n):
        # Create a sample dataset of size n by sampling with replacement from the original dataset XX.
        # Record the corresponding class labels for the sampled records for training purposes.

        sample = []  # sampled dataset
        labels = []  # class labels for the sampled records

        for _ in range(n):
            # Randomly select an index from the original dataset XX
            random_index = random.randint(0, len(XX) - 1)
            
            # Append the selected record and its label to the sample
            sample.append(XX[random_index][:-1])  # Exclude the last element (label) from the record
            labels.append(XX[random_index][-1])   # The last element is the class label

        return (sample, labels)

    def bootstrapping(self, XX):
        # Initializing the bootstap datasets for each tree
        for i in range(self.num_trees):
            data_sample, data_label = self._bootstrapping(XX, len(XX))
            self.bootstraps_datasets.append(data_sample)
            self.bootstraps_labels.append(data_label)

    def fitting(self):
        # Train `num_trees` decision trees using the bootstraps datasets and labels
        # by calling the learn function from your DecisionTree class.
        
        for i in range(self.num_trees):
            decision_tree = self.decision_trees[i]
            data_sample = self.bootstraps_datasets[i]
            data_label = self.bootstraps_labels[i]
            decision_tree.learn(data_sample, data_label)
        return None


    def voting(self, X):
        y = []

        for record in X:
            votes = []

            for i in range(len(self.bootstraps_datasets)):
                dataset = self.bootstraps_datasets[i]

                if record not in dataset:
                    OOB_tree = self.decision_trees[i]
                    effective_vote = OOB_tree.classify(record)
                    if effective_vote is not None:
                        votes.append(effective_vote)

            if len(votes) == 0:
                forest_votes = [tree.classify(record) for tree in self.decision_trees]
                non_none_votes = [vote for vote in forest_votes if vote is not None]
                if non_none_votes:
                    y.append(max(set(non_none_votes), key=non_none_votes.count))
                else:
                    y.append(None)
            else:
                counts = np.bincount(votes)
                y.append(np.argmax(counts))

        return y

    def user(self):
        """
        :return: string
        your GTUsername, NOT your 9-Digit GTId  
        """
        ### Implement your code here
        #############################################
        return 'jag31'
        #############################################

In [ ]:
#export

# TODO: Determine the forest size according to your implementation. 
# This function will be used by the autograder to set your forest size during testing
# VERY IMPORTANT: Minimum forest_size should be 10
def get_forest_size():
    forest_size = 10
    return forest_size

# TODO: Determine random seed to set for reproducibility
# This function will be used by the autograder to set the random seed to obtain the same results you achieve locally
def get_random_seed():
    random_seed = 0
    return random_seed

### Do not modify the below cell
The cell below is provided to test that your random forest classifier can be successfully built and run. Similar steps will be used to build and run your code in Gradescope. Any additional testing of functions can be done in the cells below the `%run helpers/notebook2script submission` cell, as these will not be parsed by the autograder.

In [ ]:
def run():
    np.random.seed(get_random_seed())
    # start time 
    start = datetime.now()
    X = list()
    y = list()
    XX = list()  # Contains data features and data labels
    numerical_cols = set([i for i in range(0, 31)])  # indices of numeric attributes (columns)

    # Loading data set
    print("reading the data")
    with open("Wisconsin_breast_prognostic.csv") as f:
        next(f, None)
        for line in csv.reader(f, delimiter=","):
            xline = []
            for i in range(len(line)):
                if i in numerical_cols:
                    xline.append(ast.literal_eval(line[i]))
                else:
                    xline.append(line[i])

            X.append(xline[:-1])
            y.append(xline[-1])
            XX.append(xline[:])

    # Initializing a random forest.
    randomForest = RandomForest(get_forest_size())

    # printing the name
    print("__Name: " + randomForest.user()+"__")

    # Creating the bootstrapping datasets
    print("creating the bootstrap datasets")
    randomForest.bootstrapping(XX)

    # Building trees in the forest
    print("fitting the forest")
    randomForest.fitting()

    # Calculating an unbiased error estimation of the random forest
    # based on out-of-bag (OOB) error estimate.
    y_predicted = randomForest.voting(X)

    # Comparing predicted and true labels
    results = [prediction == truth for prediction, truth in zip(y_predicted, y)]

    # Accuracy
    accuracy = float(results.count(True)) / float(len(results))

    print("accuracy: %.4f" % accuracy)
    print("OOB estimate: %.4f" % (1 - accuracy))

    # end time
    print("Execution time: " + str(datetime.now() - start))

In [ ]:
# Call the run() function to test your implementation
# Use this cell and any cells below for additional testing
run()